In [1]:
import json
import pandas as pd
import pickle
import openai
import functools
import numpy as np

# Process the Squad JSON to extract what we need

In [ ]:
d = pd.read_json('data/squad2.json')['data']

In [ ]:
all_paragraphs = [paragraph['context'] for ent in d for paragraph in ent['paragraphs']]

In [ ]:
# Ok. list comprehension is still possible but it gets a little obnoxious.
all_qa = []
paragraph_id = 0
for ent in d:
    for paragraph in ent['paragraphs']:
        for qa in paragraph['qas']:
            if len(qa['answers']) > 0 and qa['is_impossible'] == False:
                all_qa.append((paragraph_id, qa['question'], qa['answers'][0]['text']))
        paragraph_id += 1

In [ ]:
len(all_qa)

In [ ]:
len(all_paragraphs)

In [ ]:
all_paragraphs[0]

In [ ]:
all_qa[0:5]

In [ ]:
f = open('data/paragraphs_and_qa.pickle','wb')
pickle.dump(all_paragraphs, f)
pickle.dump(all_qa, f)
f.close()

# Baseline OpenAI QA. Knowing exactly the paragraphs to use

In [13]:
f = open('data/paragraphs_and_qa.pickle','rb')
all_paragraphs = pickle.load(f)
all_qa = pickle.load(f)
f.close()

In [25]:
client = openai.OpenAI(max_retries=5,timeout=10)

In [6]:
qa_cache = {}

In [7]:
def question_answerer(client, question, context):
    global qa_cache
    key = context + question
    if key in qa_cache:
        return qa_cache[key]
    system_prompt = "You are an assistant for question-answering tasks. Use the provided pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Provide just the answer in as few words as possible. Do not use complete sentences."
    user_prompt = f"Question: {question} \nContext: {context} \nAnswer:"
    response = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": user_prompt,
            }
        ],
        model="gpt-3.5-turbo",
        temperature=0.0,
    )
    response = response.choices[0].message.content
    qa_cache[key] = response
    return response

In [8]:
para_id, question, answer = all_qa[0]
print(all_paragraphs[para_id])
print(question)
print(answer)

Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".
When did Beyonce start becoming popular?
in the late 1990s


In [9]:
response = question_answerer(client, question, all_paragraphs[para_id])
print(response)

Late 1990s


## Coming with a simple comparison evaluation

In [23]:
is_same_cache = {}

In [26]:
def is_same(client, question, a1, a2):
    global is_same_cache
    key = question + ' '  +a1 + ' ' + a2
    if key in is_same_cache:
        return is_same_cache[key]

    system_prompt = "You are an assistant for scoring answers. Two answers to a hypothetical question are provided. Say 'Yes' if both answers have the same meaning, and 'No' otherwise."
    user_prompt = f"Question: {question} \Answer 1: {a1} \nAnswer 2: {a2}"
    response = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": user_prompt,
            }
        ],
        model="gpt-3.5-turbo",
        temperature=0.0
    )
    response = response.choices[0].message.content == 'Yes'
    is_same_cache[key] = response
    return response
    

In [11]:
answer

'in the late 1990s'

In [13]:
is_same(client, question, answer, response)

True

In [14]:
is_same(client, question, answer, "Late 1980s")

False

# Evaluate RAG
80k questions at about 1k tokens per question == ~ 0.0010 * 80000 or about $80. That is a bit pricy for a quick test. We will subsample to ~ 10% questions.
Note that this part is really entirely irrelevant and not that useful. But its a fun thing to try.

In [15]:
import numpy as np
sub_qa = all_qa[0:len(all_qa):10]

In [16]:
import concurrent.futures

In [17]:
def do_work(client, question, true_answer, context):
    response = question_answerer(client, question, context)
    evaluation = is_same(client, question, true_answer, response)
    return true_answer, response, evaluation

In [ ]:
import time
responses = []
evaluations = []
futures = []
ctr = 0
for ent in sub_qa:
    para_id, question, true_answer = ent
    try:
        true_answer, response, evaluation = do_work(client, question, true_answer, all_paragraphs[para_id])
    except openai.RateLimitError:
        time.sleep(60)
        true_answer, response, evaluation = do_work(client, question, true_answer, all_paragraphs[para_id])
    ctr += 1
    print(f"Correct Answer: {true_answer}, Response: {response}, Eval: {evaluation}. {ctr}/{total_futures}")
    responses.append(response)
    evaluations.append(evaluation)

In [39]:
df = pd.DataFrame()

In [40]:
df['context'] = [all_paragraphs[i[0]] for i in sub_qa]
df['question'] = [i[1] for i in sub_qa]
df['answer'] = [i[2] for i in sub_qa]
df['response'] = responses
df['evaluations'] = evaluations

In [41]:
df

,context,question,answer,response,evaluations
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,Late 1990s,True
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What was the first album Beyoncé released as a...,Dangerously in Love,Dangerously in Love,True
2,Following the disbandment of Destiny's Child i...,Which album was darker in tone from her previo...,Beyoncé,Beyoncé (2013),True
3,"A self-described ""modern-day feminist"", Beyonc...",In which decade did the Recording Industry Ass...,2000s,2000s,True
4,"Beyoncé Giselle Knowles was born in Houston, T...",Where did Beyonce get her name from?,her mother's maiden name,Her mother's maiden name.,True
...,...,...,...,...,...
8678,The Bagmati River which flows through Kathmand...,"In a Hindu funeral, who is typically the main ...",first son,The first son.,True
8679,Kirant Mundhum is one of the indigenous animis...,What is another name for ancestor worship?,worship of Ajima,Ajima worship,True
8680,"Institute of Medicine, the central college of ...",What institution of tertiary education is know...,National Academy of Medical Sciences,National Academy of Medical Sciences,True
8681,The total length of roads in Nepal is recorded...,If one wished to travel north out of Kathmandu...,Araniko,Araniko Highway,True


In [44]:
df.to_pickle('data/qa_result_df.pickle')

# OpenAI Embeddings

In [58]:
def generate_openai_embeddings(client, textlist):
    ret = []
    ctr = 0
    total = len(textlist)
    for ent in textlist:
        ctr += 1
        try:
            res = client.embeddings.create(input=ent, model='text-embedding-ada-002', encoding_format='float')
        except openai.RateLimitError:
            time.sleep(60)
            res = client.embeddings.create(input=ent, model='text-embedding-ada-002', encoding_format='float')
        ret.append(res.data[0].embedding)
        print(f"{ctr}/{total}")
    return ret

In [ ]:
par_embeddings = generate_openai_embeddings(client, all_paragraphs)

In [63]:
par_embeddings_arr = np.array(par_embeddings)

In [66]:
np.save('data/openai_paragraphs_embeddings.npy', par_embeddings_arr)

In [ ]:
qn_embeddings = generate_openai_embeddings(client, [q[1] for q in all_qa])

In [68]:
qn_embeddings_arr = np.array(qn_embeddings)

In [69]:
np.save('data/openai_qn_embeddings.npy', qn_embeddings_arr)

# Nearest Neighbor

In [2]:
from sklearn.neighbors import NearestNeighbors

In [3]:
par_embeddings_arr = np.load('data/openai_paragraphs_embeddings.npy')
qn_embeddings_arr = np.load('data/openai_qn_embeddings.npy')

In [4]:
nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree')
nbrs.fit(par_embeddings_arr)

NearestNeighbors(algorithm='ball_tree', n_neighbors=10)

In [7]:
knn = nbrs.kneighbors(qn_embeddings_arr)

In [8]:
knn

(array([[0.53342373, 0.54183945, 0.5480534 , ..., 0.55267631, 0.55477935,
         0.55652812],
        [0.51625546, 0.56791918, 0.57577417, ..., 0.60289394, 0.60317292,
         0.60357273],
        [0.47841394, 0.49049756, 0.51447069, ..., 0.54367424, 0.5441335 ,
         0.54597068],
        ...,
        [0.57681319, 0.59904399, 0.59999977, ..., 0.6253042 , 0.62548659,
         0.62892828],
        [0.49743956, 0.57099157, 0.5722643 , ..., 0.59779   , 0.60029111,
         0.60105112],
        [0.6205644 , 0.67376102, 0.68988245, ..., 0.70489665, 0.70647834,
         0.70827299]]),
 array([[   23,    45,    26, ...,    50,     0,    14],
        [    4,     5,     3, ...,    64,    42,    20],
        [    1,    11,     8, ...,     7,    23,    36],
        ...,
        [18979, 18929, 18925, ..., 18966, 18972, 18934],
        [18979, 18932, 18940, ..., 18978, 18972, 18941],
        [18979, 18255,  3350, ...,   126, 13786, 11440]]))

In [9]:
np.save('data/openai_knn_qn_to_paragraph.npy', knn)

In [10]:
_, nearest_paragraphs = knn

In [11]:
nearest_paragraphs.shape

(86821, 10)

In [14]:
truth = np.array([i[0] for i in all_qa])

In [30]:
p_at = []
p_at.append((nearest_paragraphs[:,0] == truth).sum())
for i in range(1,10):
    p = (nearest_paragraphs[:,i] == truth).sum()
    p_at.append(p_at[i-1] + p)

In [31]:
p_at

[54566, 63738, 67928, 70421, 72127, 73358, 74400, 75232, 75857, 76395]

# Simpler Embeddings

In [34]:
from rank_bm25 import BM25Okapi

In [35]:
tokenized_corpus = [par.split(" ") for par in all_paragraphs]
bm25 = BM25Okapi(tokenized_corpus)

In [69]:
bm25_nearest_paragraphs = []
ctr = 0
for qa in all_qa:
    z = bm25.get_scores(qa[1].split(" "))
    bm25_nearest_paragraphs.append(np.argsort(-z)[:10].copy())
    ctr += 1
    if ctr % 100 == 0:
        print(ctr)


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [70]:
bm25_nearest_paragraphs_arr = np.array(bm25_nearest_paragraphs)

In [72]:
np.save('data/bm25_knn_qn_to_paragraph.npy', bm25_nearest_paragraphs_arr)

In [73]:
bm25_p_at = []
bm25_p_at.append((bm25_nearest_paragraphs_arr[:,0] == truth).sum())
for i in range(1,10):
    p = (bm25_nearest_paragraphs_arr[:,i] == truth).sum()
    bm25_p_at.append(p_at[i-1] + p)

In [74]:
bm25_p_at

[42140, 61503, 67289, 70124, 72121, 73461, 74363, 75289, 75973, 76490]

# Query Augmentation

In [103]:
query_augment_cache = {}
def query_augment(client, question):
    global query_augment_cache
    key = question
    if key in query_augment_cache:
        return query_augment_cache[key]

    system_prompt = "You are to perform query augmentation for a question answering system. For the provided question, return a comma separated list of up to 5 new words which are not part of the question that can be used to improve document retrieval. These words will be added to the words already in the question and will be used to search a collection of documents to find a document which may contain the answer."
    user_prompt = f"Question: {question}"
    response = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": user_prompt,
            }
        ],
        model="gpt-3.5-turbo",
        temperature=0.0
    )
    # split by , 
    r = response.choices[0].message.content
    response = response.choices[0].message.content.split(',')
    # split by space
    response = [r.strip().split(' ') for r in response]
    # flatten
    response = [word for i in response for word in i]
    response = list(set(response))
    query_augment_cache[key] = response
    return response


In [93]:
client = openai.OpenAI(max_retries=5,timeout=10)

In [96]:
all_qa[0][1]

'When did Beyonce start becoming popular?'

In [ ]:
import time
question_augmentation = []
ctr = 0
for ent in all_qa:
    try:
        augment = query_augment(client, ent[1])
    except openai.RateLimitError:
        time.sleep(60)
        augment = query_augment(client, ent[1])
    ctr += 1
    print(f"Question: {ent[1]}, Augment: {augment}, {ctr}/{len(all_qa)}")
    question_augmentation.append(augment)

Question: When did Beyonce start becoming popular?, Augment: ['career', 'Beyonce', 'fame', 'music', 'rise'], 1/86821
Question: What areas did Beyonce compete in when she was growing up?, Augment: ['performing', 'shows', 'talent', 'dance', 'competitions', 'singing'], 2/86821
Question: When did Beyonce leave Destiny's Child and become a solo singer?, Augment: ['success', 'transition', 'augmentation', 'career', 'departure'], 3/86821
Question: In what city and state did Beyonce  grow up? , Augment: ['Houston', 'Texas', 'childhood', 'singer', 'hometown'], 4/86821
Question: In which decade did Beyonce become famous?, Augment: ['famous', 'decade', 'Beyonce', 'collection', 'become'], 5/86821
Question: In what R&B group was she the lead singer?, Augment: ['solo', 'vocalist', 'music', 'band', 'album'], 6/86821
Question: What album made her a worldwide known artist?, Augment: ['worldwide', 'artist', 'made', 'known', 'album'], 7/86821
Question: Who managed the Destiny's Child group?, Augment: ['gr

Question: I which church was Beyonce  a member and soloist  in the choir?, Augment: ['soloist', 'church', 'member', 'Beyonce', 'choir'], 57/86821
Question: What type of school was Parker Elementary School?, Augment: ['elementary', 'augmentation', 'type', 'school', 'Parker'], 58/86821
Question: Which song did Beyonce sing to win a competition at age 7?, Augment: ['7', 'competition', 'win', 'Beyonce', 'age', 'sing'], 59/86821
Question: What city was Beyoncé's elementary school located in?, Augment: ['Houston', 'Texas', "Destiny's", 'singer', 'music', 'Child'], 60/86821
Question: What was the name of Beyoncé's first dance instructor?, Augment: ['', 'first', 'instructor', 'dance', 'name', 'words:', 'Beyoncé', 'new'], 61/86821
Question: How old was Beyoncé when she won a school talent show?, Augment: ['talent', 'performance', 'won', 'show', 'Beyoncé', 'age', 'school'], 62/86821
Question: What choir did Beyoncé sing in for two years?, Augment: ['two', 'Beyoncé', 'choir', 'retrieval', 'years'